In [3]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from tqdm.auto import tqdm
from matplotlib.path import Path
import ast
import pickle
import matplotlib.pyplot as plt
from networkx.algorithms import bipartite
import networkx as nx

In [13]:
#arts = {}
#arts['space_duck']  = [[342, 1699], [338, 1703], [333, 1735], [333, 1742], [331, 1742], [330, 1754], [325, 1753], [325, 1747], [321, 1746], [316, 1745], [310, 1745], [293, 1748], [290, 1751], [290, 1755], [291, 1756], [292, 1762], [296, 1766], [303, 1766], [308, 1763], [311, 1763], [313, 1761], [319, 1761], [319, 1757], [322, 1757], [325, 1753], [330, 1753], [333, 1762], [328, 1763], [321, 1768], [316, 1771], [315, 1775], [300, 1782], [297, 1785], [296, 1791], [295, 1792], [295, 1807], [299, 1814], [304, 1815], [311, 1817], [316, 1817], [323, 1819], [326, 1820], [329, 1823], [332, 1824], [336, 1829], [342, 1830], [344, 1832], [346, 1832], [346, 1850], [311, 1850], [311, 1861], [371, 1861], [371, 1850], [346, 1850], [346, 1832], [356, 1832], [365, 1826], [369, 1822], [371, 1814], [374, 1786], [371, 1784], [371, 1782], [365, 1778], [365, 1776], [369, 1776], [376, 1766], [378, 1766], [378, 1774], [383, 1778], [385, 1779], [395, 1779], [400, 1773], [400, 1764], [396, 1760], [391, 1759], [389, 1757], [383, 1757], [379, 1762], [378, 1766], [376, 1766], [376, 1763], [378, 1760], [378, 1753], [379, 1751], [379, 1742], [378, 1741], [378, 1721], [375, 1716], [375, 1712], [371, 1705], [362, 1700], [360, 1699], [341, 1699]]
space_duck  = [[342, 1699], [338, 1703], [333, 1735], [333, 1742], [331, 1742], [330, 1754], [325, 1753], [325, 1747], [321, 1746], [316, 1745], [310, 1745], [293, 1748], [290, 1751], [290, 1755], [291, 1756], [292, 1762], [296, 1766], [303, 1766], [308, 1763], [311, 1763], [313, 1761], [319, 1761], [319, 1757], [322, 1757], [325, 1753], [330, 1753], [333, 1762], [328, 1763], [321, 1768], [316, 1771], [315, 1775], [300, 1782], [297, 1785], [296, 1791], [295, 1792], [295, 1807], [299, 1814], [304, 1815], [311, 1817], [316, 1817], [323, 1819], [326, 1820], [329, 1823], [332, 1824], [336, 1829], [342, 1830], [344, 1832], [346, 1832], [346, 1850], [311, 1850], [311, 1861], [371, 1861], [371, 1850], [346, 1850], [346, 1832], [356, 1832], [365, 1826], [369, 1822], [371, 1814], [374, 1786], [371, 1784], [371, 1782], [365, 1778], [365, 1776], [369, 1776], [376, 1766], [378, 1766], [378, 1774], [383, 1778], [385, 1779], [395, 1779], [400, 1773], [400, 1764], [396, 1760], [391, 1759], [389, 1757], [383, 1757], [379, 1762], [378, 1766], [376, 1766], [376, 1763], [378, 1760], [378, 1753], [379, 1751], [379, 1742], [378, 1741], [378, 1721], [375, 1716], [375, 1712], [371, 1705], [362, 1700], [360, 1699], [341, 1699]]
pufferfish_pond = [[1999, 1118], [1870, 1120], [1870, 917], [1999, 917]]
france = [[125, 497], [175, 497], [175, 288], [171, 284], [171, 275], [148, 275], [148, 300], [141, 307], [141, 320], [152, 320], [152, 328], [125, 328], [125, 345], [132, 345], [132, 386], [125, 386]]
butt = [[185, 1928], [183, 1602], [76, 1600], [75, 1930]]
battle_heroes = [[683, 1510], [710, 1553], [707, 1555], [707, 1571], [707, 1573], [697, 1574], [675, 1570], [673, 1591], [638, 1595], [627, 1604], [628, 1678], [733, 1684], [732, 1691], [724, 1696], [718, 1702], [716, 1714], [720, 1726], [728, 1733], [742, 1734], [754, 1728], [758, 1715], [757, 1703], [751, 1695], [741, 1690], [740, 1691], [739, 1684], [750, 1682], [750, 1670], [779, 1669], [786, 1674], [787, 1711], [830, 1713], [829, 1706], [834, 1705], [833, 1697], [843, 1695], [837, 1660], [831, 1652], [831, 1632], [808, 1609], [812, 1601], [810, 1592], [796, 1586], [789, 1586], [760, 1590], [752, 1585], [751, 1560], [758, 1558], [758, 1544], [753, 1545], [752, 1534], [745, 1541], [743, 1547], [733, 1565], [730, 1565], [726, 1563], [723, 1561], [698, 1521], [685, 1510]]

In [16]:
arts = [space_duck,pufferfish_pond,france,butt,battle_heroes]
arts[0]

[[342, 1699],
 [338, 1703],
 [333, 1735],
 [333, 1742],
 [331, 1742],
 [330, 1754],
 [325, 1753],
 [325, 1747],
 [321, 1746],
 [316, 1745],
 [310, 1745],
 [293, 1748],
 [290, 1751],
 [290, 1755],
 [291, 1756],
 [292, 1762],
 [296, 1766],
 [303, 1766],
 [308, 1763],
 [311, 1763],
 [313, 1761],
 [319, 1761],
 [319, 1757],
 [322, 1757],
 [325, 1753],
 [330, 1753],
 [333, 1762],
 [328, 1763],
 [321, 1768],
 [316, 1771],
 [315, 1775],
 [300, 1782],
 [297, 1785],
 [296, 1791],
 [295, 1792],
 [295, 1807],
 [299, 1814],
 [304, 1815],
 [311, 1817],
 [316, 1817],
 [323, 1819],
 [326, 1820],
 [329, 1823],
 [332, 1824],
 [336, 1829],
 [342, 1830],
 [344, 1832],
 [346, 1832],
 [346, 1850],
 [311, 1850],
 [311, 1861],
 [371, 1861],
 [371, 1850],
 [346, 1850],
 [346, 1832],
 [356, 1832],
 [365, 1826],
 [369, 1822],
 [371, 1814],
 [374, 1786],
 [371, 1784],
 [371, 1782],
 [365, 1778],
 [365, 1776],
 [369, 1776],
 [376, 1766],
 [378, 1766],
 [378, 1774],
 [383, 1778],
 [385, 1779],
 [395, 1779],
 [400,

In [19]:
df = pd.read_csv("data/artworks_ordered.csv")
df[:5]

,Unnamed: 0.1,Unnamed: 0,id,name,path,center,area
0,5372,5372,txrg84,Flag of France,"{'109-164, T': [[249, 1469], [0, 1469], [0, 19...","{'109-164, T': [125, 1719]}",124500.0
1,1187,1187,twpyje,Flag of Germany,"{'109-166, T': [[0, 1121], [448, 1121], [447, ...","{'109-166, T': [992, 1148]}",96223.0
2,258,258,000354,The Nordic Union,"{'1-166, T': [[187, 93], [187, 97], [207, 97],...","{'1-166, T': [506, 129]}",89953.5
3,4866,4866,txd8wt,Central Alliance,"{'1-166, T': [[448, 890], [512, 890], [512, 90...","{'1-166, T': [447, 994]}",75325.5
4,7916,7916,ubyhtf,Netherlands,"{'56-166, T': [[297, 13], [297, 35], [886, 35]...","{'56-166, T': [1171, 19]}",54044.0


In [11]:
df.iloc[0]

Unnamed: 0.1                                                 5372
Unnamed: 0                                                   5372
id                                                         txrg84
name                                               Flag of France
path            {'109-164, T': [[249, 1469], [0, 1469], [0, 19...
center                                {'109-164, T': [125, 1719]}
area                                                     124500.0
Name: 0, dtype: object

In [5]:
NUM_ARTS = 5
df = df[:NUM_ARTS]

NameError: name 'df' is not defined

In [21]:
artworks = {}
id_count = 1
for row in df.itertuples():
    name = row.name
    path = row.path
    path = ast.literal_eval(path)
    coords = list(path.values())[0]
    artworks[id_count] = coords
    id_count += 1

In [18]:
artworks = {}
id_count = 1
for row in arts:
    coords = row
    artworks[id_count] = coords
    id_count += 1

In [17]:
# Flag of France
#art1 = [[125, 497], [175, 497], [175, 288], [171, 284], [171, 275], [148, 275], [148, 300], [141, 307], [141, 320], [152, 320], [152, 328], [125, 328], [125, 345], [132, 345], [132, 386], [125, 386]]
#flag of argentina
#art2 = [[1533, 1819], [1773, 1819], [1774, 1917], [1532, 1916]]

In [9]:
def find_points(art):
    x, y = np.meshgrid(np.arange(2000), np.arange(2000)) # make a canvas with coordinates
    x, y = x.flatten(), y.flatten()
    points = np.vstack((x,y)).T 

    p = Path(art) # make a polygon
    grid = p.contains_points(points,radius = 0)
    mask = grid.reshape(2000,2000) # now you have a mask with points inside a polygon
    x,y = mask.nonzero()
    coords = []
    for xc,yc in zip(x,y):
        coords.append((xc,yc))
    return coords

In [ ]:
CHUNK_SIZE = 3_000_000
MIN_TILES = 2

f = open('./output/bipartite_network_' + str(MIN_TILES) + '_'+ str(NUM_ARTS) + '.csv', 'w')
counter = 0

for key in tqdm(artworks.keys()):
    art_pixel = find_points(artworks[key])
    s = set(art_pixel)
    name = key
    user_actions = {}
    l = []

    with pd.read_csv(
            'reddit_place_2022_trimmed.csv',
            chunksize=CHUNK_SIZE,
            engine="c",
            dtype={'timestamp':np.uint32,
              'user_id':np.uint32,
              'pixel_color':np.uint8,
              'x':np.uint16,
              'y':np.uint16,}
        ) as csv:
            for chunk in tqdm(csv):
                for row in chunk.itertuples():
                    user = row.user_id
                    x = row.x
                    y = row.y
                    if (x,y) in s:
                        if  user in user_actions:
                            user_actions[user] = user_actions[user] + 1 
                        else:
                            user_actions[user] = 1
               
    for user_key in user_actions.keys():
        if user_actions[user_key] >= MIN_TILES:
            f.write(str(name) + ' ' + str(user_key) + '\n')
            l.append(user_actions[user_key])
            counter +=  1
            
    pickle.dump(l, open('./output/tiles/distrib_tiles_' + str(name) + '.pickle', "wb")) 
    print(counter)
f.close()

# weighted bipartite sets

In [19]:
CHUNK_SIZE = 3_000_000
MIN_TILES = 5

f = open('./output/bipartite_network_weighted_' + str(MIN_TILES) + '_'+ str(NUM_ARTS) + '.csv', 'w')
counter = 0

for key in tqdm(artworks.keys()):
    art_pixel = find_points(artworks[key])
    s = set(art_pixel)
    name = key
    user_actions = {}
    l = []

    with pd.read_csv(
            'reddit_place_2022_trimmed.csv',
            chunksize=CHUNK_SIZE,
            engine="c",
            dtype={'timestamp':np.uint32,
              'user_id':np.uint32,
              'pixel_color':np.uint8,
              'x':np.uint16,
              'y':np.uint16,}
        ) as csv:
            for chunk in tqdm(csv):
                for row in chunk.itertuples():
                    user = row.user_id
                    x = row.x
                    y = row.y
                    if (x,y) in s:
                        if  user in user_actions:
                            user_actions[user] = user_actions[user] + 1 
                        else:
                            user_actions[user] = 1
               
    for user_key in user_actions.keys():
        if user_actions[user_key] > MIN_TILES:
            f.write(str(name) + ' ' + str(user_key) + ' ' + str(user_actions[user_key]) + '\n')
            l.append(user_actions[user_key])
            counter +=  1
            
    pickle.dump(l, open('./output/tiles/distrib_tiles_' + str(name) + '.pickle', "wb")) 
    print(counter)
f.close()

  0%|          | 0/5 [00:00<?, ?it/s]

0it [00:00, ?it/s]

7323


0it [00:00, ?it/s]

23010


...

35761


0it [00:00, ?it/s]

66763


0it [00:00, ?it/s]

99894
